<a href="https://colab.research.google.com/github/KhoomeiK/MindMapResearch/blob/master/HAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DATA PREP STUFF

In [0]:
# download dataset and labels
! pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# downloaded1 = drive.CreateFile({'id': '1oZb283stxpZn8Dn8i8e2Vh6P8d6Voj4Y'}) 
# downloaded1.GetContentFile('dataset.zip')  

# ! unzip dataset.zip
# ! rm -rf cse198f_shiv/data
# ! rm -rf cse198f_shiv/diagnostics
# ! rm -rf cse198f_shiv/models
# ! rm cse198f_shiv/vectors.py
# ! ls cse198f_shiv

downloaded2 = drive.CreateFile({'id': '1-1nQU2lUwBnEyNot0EeVK72X92bdqVAu'}) 
downloaded2.GetContentFile('labels.pkl')

downloaded = drive.CreateFile({'id': '1-XYU2MCNbhS8ir_7ToC5DvrUMsQy9-9E'}) 
downloaded.GetContentFile('embeddings.zip')

! unzip embeddings.zip

Archive:  embeddings.zip
   creating: embeddings/
  inflating: embeddings/_astronautmikedexter.pkl  
  inflating: embeddings/buttpooperson.pkl  
  inflating: embeddings/BTAUB.pkl    
  inflating: embeddings/_Hellchic_.pkl  
  inflating: embeddings/Campaschristmas.pkl  
  inflating: embeddings/_leftoverspaghetti.pkl  
  inflating: embeddings/Botslavia.pkl  
  inflating: embeddings/CaptainBathrobe.pkl  
  inflating: embeddings/7Dublinah.pkl  
  inflating: embeddings/0berfeld.pkl  
  inflating: embeddings/bow_m0nster.pkl  
  inflating: embeddings/_Aggort.pkl  
  inflating: embeddings/britishdude1984.pkl  
  inflating: embeddings/buttbugle.pkl  
  inflating: embeddings/18b36.pkl    
  inflating: embeddings/BPDprisoner.pkl  
  inflating: embeddings/BShears.pkl  
  inflating: embeddings/2crowsonmymantle.pkl  
  inflating: embeddings/cant-live-this-life.pkl  
  inflating: embeddings/cannarchista.pkl  
  inflating: embeddings/3AMThrowaway_.pkl  
  inflating: embeddings/BucketsMcGaughey.pkl  
 

In [0]:
# read dataset into memory
from os import listdir
from os.path import isfile, join
import pandas as pd

mypath = 'cse198f_shiv'
csvs = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(len(csvs))

data = []
names = []
for csv in csvs:
    if csv[-4:] == '.csv':
        try:
            data.append(pd.read_csv(join(mypath, csv), encoding='CP1252'))
            names.append(csv[:-4])
        except:
            try:
                data.append(pd.read_csv(join(mypath, csv), encoding='UTF8'))
                names.append(csv[:-4])
            except:
                continue
print(len(data))

# pd.reset_option('all')
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

2052
1980


In [0]:
# # download embedding tools
# ! ls *
# ! mkdir fastText
# ! curl https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip > fastText/crawl-300d-2M.vec.zip
# ! unzip fastText/crawl-300d-2M.vec.zip -d fastText/
# ! mkdir encoder
# ! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
# ! curl https://raw.githubusercontent.com/facebookresearch/InferSent/master/models.py > models.py
# ! mkdir embeddings

In [0]:
# generate embeddings of dataset
import torch, os
import numpy as np
import tensorflow_hub as hub

from absl import logging
from models import InferSent

import pickle
import time

MODEL_PATH = 'encoder/infersent2.pkl'
W2V_PATH = 'fastText/crawl-300d-2M.vec'

def load_infersent_model(model_path=MODEL_PATH, word_embeddings_path=W2V_PATH):
	params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048, 'pool_type': 'max', 'dpout_model': 0.0, 'version': 2}
	infersent = InferSent(params_model).cuda()
	infersent.load_state_dict(torch.load(model_path))
	infersent.set_w2v_path(word_embeddings_path)
	infersent.build_vocab_k_words(K=100000)
	return infersent

def get_infersent_vectors(sentences, model):
	return model.encode(sentences, tokenize=False, verbose=False)

def get_user_data_embeddings(comments, model):
	# model = load_infersent_model()
	embedding = get_infersent_vectors(comments, model)
	return embedding

dataEmbeddings = []
model = load_infersent_model()

start = time.time()
for i, name in enumerate(names):
	comments = list(data[i]['text']) if 'text' in data[i] else []
	if len(comments) >= 1:
		print(i, name)
		try:
			embeddings = get_user_data_embeddings(data[i]['text'], model)
			print(len(embeddings), 'comments')
			dataEmbeddings.append(embeddings)
			with open('embeddings/%s.pkl' % name, 'wb') as pkl:
				pickle.dump(embeddings, pkl)
		except:
			print('ERROR')

print(time.time() - start)

## MODEL RUN

In [0]:
# load labels and user embeddings and create Users data obj
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from csv import reader
import pickle
from os import listdir
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Users(Dataset):
    def __init__(self, embedPath, labelPath, split): 
        self.users, self.labels = [], []
        users = {}
        labels = pickle.load(open(labelPath, 'rb')) # { username: [ depressionPercent, vaderScore ] }
        fileList = listdir(embedPath)

        if type(split) == float:
            userList = random.sample(fileList, int(len(fileList) * split))
            for user in userList: # must download and unzip embeddings.zip first
                users[user[:-4]] = pickle.load(open('%s/%s' % (embedPath, user), 'rb'))
            
            intersection = list(set(users.keys()).intersection(set(labels.keys())))
            for i in intersection:
                userTensor = torch.tensor(users[i])
                if list(userTensor.shape)[0] != 1: # exclude single comment users
                    self.users.append(userTensor)
                    self.labels.append(torch.tensor(labels[i]))

            self.usernames = users.keys()
    
        elif type(split) == Users:
            for user in fileList: # must download and unzip embeddings.zip first
                if user[:-4] not in split.usernames:
                    users[user[:-4]] = pickle.load(open('%s/%s' % (embedPath, user), 'rb'))
            
            intersection = list(set(users.keys()).intersection(set(labels.keys())))
            for i in intersection:
                userTensor = torch.tensor(users[i])
                if list(userTensor.shape)[0] != 1: # exclude single comment users
                    self.users.append(userTensor)
                    self.labels.append(torch.tensor(labels[i] * 100))
        
        else:
            print('ERROR')

        print(len(self), self[0])

    def __getitem__(self, i):
        return self.users[i], self.labels[i][0] * 100 # remove [0]
    
    def __len__(self):
        assert len(self.users) == len(self.labels)
        return len(self.users)

# def my_collate(batch):
#     data = [item[0] for item in batch]
#     target = [item[1] for item in batch]
#     target = torch.Tensor(target)
#     return [data, target]

train = Users('embeddings', 'labels.pkl', 0.8)
test = Users('embeddings', 'labels.pkl', train)
dataloader_params = {'shuffle': True}
# dataloader_params = {'batch_size': 4, 'shuffle': True, 'collate_fn': my_collate}
# train, test = DataLoader(train, **dataloader_params), DataLoader(test, **dataloader_params)

1385 (tensor([[ 7.4689e-03, -2.7654e-02,  5.2724e-02,  ..., -2.0560e-02,
         -2.0402e-02, -2.5424e-02],
        [ 7.4689e-03, -7.5594e-02,  4.1906e-02,  ...,  4.9380e-02,
          2.9996e-02, -1.3509e-02],
        [ 7.4689e-03,  1.1295e-03,  9.3069e-02,  ...,  4.1665e-02,
          4.5500e-03,  3.3463e-03],
        ...,
        [ 7.4689e-03, -3.9357e-02, -2.8357e-02,  ...,  3.8663e-02,
          1.5528e-02, -6.5598e-03],
        [ 7.4689e-03, -2.7818e-02, -2.2223e-02,  ..., -9.0610e-04,
         -7.4943e-03, -5.2340e-05],
        [ 7.4689e-03, -2.8382e-02, -7.1140e-02,  ..., -4.1884e-02,
          2.7456e-02, -2.3299e-02]]), tensor(0.4151))
348 (tensor([[ 0.0075,  0.0788,  0.1174,  ..., -0.0365,  0.0399, -0.0098],
        [ 0.0075, -0.0661,  0.0736,  ..., -0.0027,  0.0052, -0.0084],
        [ 0.0075, -0.0278,  0.0935,  ...,  0.0078,  0.0007, -0.0120],
        ...,
        [ 0.0075, -0.0903,  0.1261,  ..., -0.0092, -0.0249, -0.0242],
        [ 0.0075, -0.0304,  0.0830,  ...,  0.01

In [0]:
def matrix_mul(input, weight, bias=False):
    feature_list = []
    _input = input.squeeze(0)
    for feature in _input:
        feature = feature.unsqueeze(1).reshape(1, 300)
        feature = torch.mm(feature, weight)
        if isinstance(bias, torch.nn.parameter.Parameter):
            feature = feature + bias.expand(feature.size()[0], bias.size()[1])
        feature = torch.tanh(feature)#.unsqueeze(0) ?
        feature_list.append(feature)

    return torch.cat(feature_list, 0)#.squeeze(1) ?

def element_wise_mul(input1, input2):
    _input1 = input1.squeeze(0)
    feature_list = []
    for feature_1, feature_2 in zip(_input1, input2):
        feature = feature_1 * feature_2
        feature_list.append(feature.unsqueeze(0))

    output = torch.cat(feature_list, 0)
    return torch.sum(output, 0).unsqueeze(0)

class AN(nn.Module):
    def __init__(
        self,
        batch_size=4,
        embedding_dimension=4096, # from glove to infersent?
        hidden_size=150, 
        n_layers=1, # multiple?
    ):
        super(AN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.sent_weight = nn.Parameter(torch.randn(2 * hidden_size, 2 * hidden_size))
        self.sent_bias = nn.Parameter(torch.randn(1, 2 * hidden_size))
        self.context_weight = nn.Parameter(torch.randn(2 * hidden_size, 1))

        self.gru = nn.GRU(embedding_dimension, hidden_size, bidirectional=True)
        self.fc = nn.Linear(2 * hidden_size, 1) 
        self._create_weights(mean=0.005)

    def _create_weights(self, mean=0.0, std=0.01):
        self.sent_weight.data.normal_(mean, std)
        self.sent_bias.data.normal_(mean, std)
        self.context_weight.data.normal_(mean, std)

    def forward(self, user):
        f_output, h_output = self.gru(user)
        output = matrix_mul(f_output, self.sent_weight, self.sent_bias)
        output = matrix_mul(output, self.context_weight)#.permute(1, 0) ?
        output = F.softmax(output)
        output = element_wise_mul(f_output, output).squeeze(0) # '''output.permute(1, 0) ?'''
        output = self.fc(output)
        output = F.leaky_relu(output)

        return output #, h_output

class Simple(nn.Module):
    def __init__(
        self,
        batch_size=4,
        embedding_dimension=4096,
        hidden_size=150, 
    ):
        super(Simple, self).__init__()
        self.batch_size = batch_size
        self.hidden_size = hidden_size

        self.gru = nn.GRU(embedding_dimension, hidden_size, bidirectional=True)
        self.fc = nn.Linear(2 * hidden_size, 1) 

    def forward(self, user):
        f_output, h_output = self.gru(user)
        output = f_output[0][-1]
        output = self.fc(output)
        output = F.leaky_relu(output)

        return output #, h_output

In [0]:
# ! pip install skorch
from sklearn.datasets import make_classification
from skorch import NeuralNetClassifier

# X, Y = [], []
# for x, y in train:
#     X.append(x.numpy())
#     Y.append(y.numpy())

hidden_size = 150
learning_rate = 0.0001
epochs = 10

net = NeuralNetClassifier(
    Simple(hidden_size=hidden_size),
    lr=learning_rate,
    max_epochs=epochs,
    iterator_train__shuffle=True,
)

print('ready to fit')
print(Y)
net.fit(X, Y) # IndexError: too many indices for array

# from sklearn.model_selection import RandomizedSearchCV

# random_grid = {'learning_rate': learning_rate,
#                'hidden_size': hidden_size,
#                'epochs': epochs}
# print(random_grid)

# rf_random = RandomizedSearchCV(estimator=net, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs = -1)
# rf_random.fit(X, Y)
# # rf.score(X_test, y_test)

# rf_random.best_params_

ready to fit
[array(0.41505992, dtype=float32), array(0.25681177, dtype=float32), array(0.270636, dtype=float32), array(0.25757578, dtype=float32), array(0.45266092, dtype=float32), array(0.27917963, dtype=float32), array(0.46417174, dtype=float32), array(0.3567279, dtype=float32), array(0.26112065, dtype=float32), array(0.44014087, dtype=float32), array(0.44546673, dtype=float32), array(0.42024833, dtype=float32), array(0.43724552, dtype=float32), array(0.4014073, dtype=float32), array(0.51781905, dtype=float32), array(0.4458877, dtype=float32), array(0.46738875, dtype=float32), array(0.45391876, dtype=float32), array(0.43733302, dtype=float32), array(0.3742865, dtype=float32), array(0.36844626, dtype=float32), array(0.2972772, dtype=float32), array(0.43357617, dtype=float32), array(0.46228343, dtype=float32), array(0.4523212, dtype=float32), array(0.4074491, dtype=float32), array(0.32730073, dtype=float32), array(0.24696779, dtype=float32), array(0.41163814, dtype=float32), array(0.4

IndexError: ignored

In [0]:
model = Simple().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()
epochs = 10
model.train()

train_losses = []
for epoch in range(epochs):
    total = 0
    losses = []
    for X, Y in train:
        # loss = 0
        # X, Y = [x.unsqueeze(0).to(device) for x in X], [y.to(device) for y in Y]
        # for x, y in zip(X, Y):
        #     pred = model(x)
        #     # print(pred, y)
        #     if loss == 0:
        #         loss = criterion(pred, y)
        #     else:
        #         loss += criterion(pred, y)
        # loss /= 4
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        loss = criterion(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        total += 1

        if total % 300 == 0:
            print(pred.item(), Y.item(), loss.item())
    
    epoch_loss = sum(losses) / total
    print(epoch, epoch_loss)
    train_losses.append(epoch_loss)

print(train_losses)

In [0]:
from matplotlib import pyplot as plt

plt.xticks(range(len(train_losses)))
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(train_losses, '-ro')

torch.save(model.state_dict(), 'model_save.pkl')
upload = drive.CreateFile({'title': 'model_save.pkl'})
upload.SetContentFile('model_save.pkl')
upload.Upload()

In [0]:
torch.no_grad()
total = 0
losses = []

for X, Y in test:
    # X, Y = [x.unsqueeze(0).to(device) for x in X], [y.to(device) for y in Y]
    # for x, y in zip(X, Y):
    #     pred = model(x)
    #     loss = criterion(pred, y)
        
    #     # print('pred', pred.item())
    #     # print('targ', y.item())
    #     # print('loss', loss.item())
        
    #     total += 1
    #     losses.append(loss.item())

    X, Y = X.to(device), Y.to(device)
    pred = model(X)
    loss = criterion(pred, Y)

    losses.append(loss.item() if loss.item() < 1 else 0)
    total += 1
    
    # if total % 20 == 0:
    if loss.item() > 0.1:
        print(pred.item(), Y.item(), loss.item())

test_loss = sum(losses) / total
print(test_loss)